# Check Exposure list from Butler (saved in csv file)

- author : Sylvie Dagoret-Campagne
- creation date : 2025-09-15
- last update : 2025-09-16
- last update : 2025-09-18 : check both repos
- last update : 2025-09-19 : add Target
- last update : 2025-09-20 : Change path of inputs
- last update : 2025-10-19 : Add cumulative counts
- last update : 2025-10-23 : png figures
- last update : 2025-10-25 : add last october data

- read the list of exposure generated by TOOL_ListofExposures.ipynb
- plot the different filters vs time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"


In [ ]:
# where are stored the figures
pathfigs = "figs_CHECK00"
prefix = "check00"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
FLAG_REPO_EMBARGO = False
if FLAG_REPO_EMBARGO:
    repo="/repo/embargo"
else:
    repo="/repo/main"
reponame = repo.replace("/","_")

path_exposureslist = "data/butlerregistry"

# when 2025 data where in /repo/embargo and 2022-2024 data in /repo/main
creation_date = "2025-09-20"
creation_date = "2025-09-21"

# all data 2022-2025 have been moved to /repo/main
#creation_date = "2025-10-17"
creation_date = "2025-10-25"
listexposures_file = os.path.join(path_exposureslist,f"{creation_date}_holosummary_all_filters" + reponame + ".csv")

In [ ]:
the_suptitle = f"Butler : {reponame}"

In [ ]:
df = pd.read_csv(listexposures_file,index_col=0)
df = df.reset_index(drop=True)

In [ ]:
df["nightObs"] = df.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df

In [ ]:
# conversion en datetime
df["time_start"] = pd.to_datetime(df["time_start"])
df["time_end"]   = pd.to_datetime(df["time_end"])

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

## dependance with filter time

In [ ]:
df["filter"].unique()

### Plot sequence number vs date for different filters

In [ ]:
plt.figure(figsize=(20,8))

# conversion en datetime
df["date"] = pd.to_datetime(df["day_obs"].astype(str), format="%Y%m%d")


sns.scatterplot(
    data=df, 
    x="date",       # abscisse en datetime
    y="seq_num",    # ou ra, dec, etc.
    hue="filter", 
    palette="tab10"
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Date of observation")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)  # lisibilité des dates
#plt.legend(loc="upper left",ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



plt.figure(figsize=(20,8))
sns.scatterplot(
    data=df,
    x="time_start",   # abscisse
    y="seq_num",      # tu peux mettre ra/dec si tu préfères
    hue="filter",
    palette="tab10"
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Time")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)
#plt.legend(loc="upper left",ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(20,8))
sns.scatterplot(
    data=df,
    x="time_start",
    y="seq_num",
    hue="filter",
    palette="Set2",       # palette plus contrastée
    s=30,                # taille des points
    edgecolor="black",    # contour noir
    linewidth=0.2
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Time")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)
#plt.legend(loc="upper left", ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.tight_layout()
plt.grid()
plt.show()


### Plot filter type vs date for different filters

In [ ]:
plt.figure(figsize=(20,8))
sns.scatterplot(
    data=df,
    x="time_start",
    y="filter",    # chaque catégorie a une ligne
    hue="filter",
    palette="Set1",
    s=100,
    edgecolor="black",
    linewidth=0.2
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})")
plt.xlabel("Time")
plt.ylabel("Filter")
plt.xticks(rotation=45)
plt.tight_layout()
#plt.legend(ncols=5)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.grid()
plt.show()


In [ ]:

# Compter le nombre d’entrées par nightObs et FILTER
counts = df[df.nightObs>20250101].groupby(["nightObs", "filter"]).size().unstack(fill_value=0)

# Plot en barres empilées
counts.plot(kind="bar", stacked=False, figsize=(18,6))

plt.ylabel("Nombre d'entrées")
plt.xlabel("nightObs")
plt.title("Nombre d'entrées par FILTER et par nightObs")
plt.legend(title="FILTER")
plt.suptitle(the_suptitle)
plt.tight_layout()
plt.show()


### Plot filter type vs date for different filters

In [ ]:
import numpy as np

# on décale légèrement la position en y en fonction de seq_num
df["filter_seq"] = df["filter"].astype(str) + "_" + df["seq_num"].astype(str)

plt.figure(figsize=(20,8))
sns.stripplot(
    data=df,
    x="time_start",
    y="filter",
    hue="filter",
    palette="Set1",
    size=4,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0
)

plt.title(f"Auxtel Holo observations wrt date and filter type ({reponame})",fontsize=15)
plt.xlabel("Time")
plt.ylabel("Filter")
plt.xticks(rotation=45)
#plt.legend(loc="upper left", ncol=6)
plt.grid()
plt.suptitle(the_suptitle)
plt.tight_layout()
plt.show()


## Plot Target vs date

In [ ]:
plt.figure(figsize=(20,16))
df["target_seq"] = df["target"].astype(str) + "_" + df["seq_num"].astype(str)
sns.stripplot(
    data=df,
    x="time_start",
    y="target",
    hue="target",
    palette="Set2",
    size=10,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0,
    edgecolor="black",
    linewidth=0.1
)
plt.title(f"Auxtel Holo observations wrt date and target ({reponame})",fontsize=15)
plt.xlabel("Time")
plt.ylabel("Target")
plt.xticks(rotation=45)
plt.suptitle(the_suptitle)
plt.tight_layout()
plt.grid()
plt.show()

## Plot cumulative sums over time 

### Cumulative sum for all filters individually

### Cumulative sum

In [ ]:
nobs = len(df)
nobs

In [ ]:
# Compter 1 par exposition
df['n_exposures'] = 1

# Calculer la somme cumulative globale
df_sorted = df.sort_values('time_start')
df_sorted['total_cumsum'] = df_sorted['n_exposures'].cumsum()

# Calculer la somme cumulative par filtre
cumsum_by_filter = (
    df_sorted
    .groupby(['filter', 'time_start'])['n_exposures']
    .sum()
    .groupby(level=0)
    .cumsum()
    .reset_index()
)

# --- Plot ---

plt.figure(figsize=(10, 6))

# Total cumulative exposures
plt.plot(df_sorted['time_start'], df_sorted['total_cumsum'],
         label='Total exposures', color='black', linewidth=2)

# Cumulative exposures per filter
for f in cumsum_by_filter['filter'].unique():
    data = cumsum_by_filter[cumsum_by_filter['filter'] == f]
    plt.plot(data['time_start'], data['n_exposures'],
             label=f'Filter: {f}', linewidth=1.8)

plt.xlabel('Observation date', fontsize=12)
plt.ylabel('Cumulative number of exposures', fontsize=12)
plt.title(f"Cumulative Exposures Over Time (not = {nobs})", fontsize=14)
plt.legend(title='Legend', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(rotation=45, ha='right')
plt.suptitle(the_suptitle)
plt.tight_layout()
figname =f"{pathfigs}/{prefix}_cumsumallfilters"+figtype
plt.savefig(figname)
plt.show()


### Cumulative sum for all filters regroupped by their colors

#### Define filters to remove

In [ ]:
# Remove unwanted filters
filters_to_remove = ['cyl_lens~holo4_001', 
                     'empty~holo4_001', 
                     'cyl_lens~holo4_004', 
                     'empty~holo4_001', 
                     'collimator~holo4_003', 
                     'SDSSr~holo4_003', 
                     'unknown~holo4_003',
                     'FELH0600~holo4_003'
                    ]
df = df[~df['filter'].isin(filters_to_remove)]

In [ ]:
# Simplify filter categories
def simplify_filter(f):
    if f.startswith('OG550'):
        return 'OG550'
    elif f.startswith('BG40'):
        return 'BG40'
    elif f.startswith('empty'):
        return 'empty'
    else:
        return 'other'  # In case something unexpected remains

In [ ]:
df['filter_group'] = df['filter'].apply(simplify_filter)

In [ ]:
# Keep only the three desired categories
df = df[df['filter_group'].isin(['empty', 'OG550', 'BG40'])]

# Add one exposure per row
df['n_exposures'] = 1

# Sort by time
df = df.sort_values('time_start')

In [ ]:
nobs = len(df)
nobs

#### Plot cumulative sum using color families

In [ ]:

# Global cumulative sum
df['total_cumsum'] = df['n_exposures'].cumsum()

# Cumulative sum by filter group
cumsum_by_filter = (
    df.groupby(['filter_group', 'time_start'])['n_exposures']
    .sum()
    .groupby(level=0)
    .cumsum()
    .reset_index()
)

In [ ]:
# Choose a colormap and sample 4 colors evenly spaced
colors = plt.cm.Dark2.colors[:4]  # or use 'Set2', 'Dark2', 'tab20', etc.
print(colors)

In [ ]:
palette = plt.cm.Dark2.colors[:4]  # Soft natural colors

color_map = {
    'empty': palette[0],
    'OG550': palette[1],
    'BG40': palette[2],
    'Total exposures': palette[3],
}

color_map = {
    'empty': "green",
    'OG550': "red",
    'BG40': "blue",
    'Total exposures': "k",
}


In [ ]:

# --- Plot ---
plt.figure(figsize=(10, 6))

# Total cumulative exposures (black)
#plt.plot(df['time_start'], df['total_cumsum'],
#         label='Total exposures', color='black', linewidth=2)

# Example use:
plt.plot(df['time_start'], df['total_cumsum'],
         label='Total exposures', color=color_map['Total exposures'], linewidth=2)


# Per filter group
#for f in cumsum_by_filter['filter_group'].unique():
#    data = cumsum_by_filter[cumsum_by_filter['filter_group'] == f]
#    plt.plot(data['time_start'], data['n_exposures'],
#             label=f'Filter: {f}', linewidth=2)

for f in cumsum_by_filter['filter_group'].unique():
    data = cumsum_by_filter[cumsum_by_filter['filter_group'] == f]
    plt.plot(data['time_start'], data['n_exposures'],
             label=f'Filter: {f}', color=color_map[f], linewidth=2)



plt.xlabel('Observation date', fontsize=12)
plt.ylabel('Cumulative number of exposures', fontsize=12)
plt.title(f"Cumulative Exposures Over Time by Filter Group (tot = {nobs})", fontsize=14)
plt.legend(title='Filter group', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)
# Rotate x-axis labels
plt.xticks(rotation=45, ha='right')
plt.suptitle(the_suptitle)
plt.tight_layout()
figname =f"{pathfigs}/{prefix}_cumsumcolorfilters"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
df.tail(20)